In [9]:
def process_prescales_data(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # Prepare to collect data
    current_run_data = []
    consolidated_data = []

    # Process each line
    for line in lines:
        if line.strip().startswith('+'):
            if current_run_data:
                # Join all collected data for the current run into a single line
                consolidated_data.append(','.join(current_run_data))
                current_run_data = []  # Reset for the next run
            continue  # Skip the header separator lines

        # Remove the leading and trailing '|' and split by '|'
        parts = line.strip().strip('|').split('|')
        # Clean up whitespace around each part
        parts = [part.strip() for part in parts]

        # Check if it's a continuation line or a new run line
        if parts[0]:  # This means it's a new run line
            if current_run_data:
                # Append the current run data before starting a new run
                consolidated_data.append(','.join(current_run_data))
                current_run_data = []
            # Initialize with the first part of the data
            current_run_data = parts[:6]  # Assuming the first 6 columns are fixed

        # Append the l1bit/prescval part to the last element
        current_run_data.append(parts[-1])  # Use the last part which should be l1bit/prescval

    # Don't forget to add the last run's data
    if current_run_data:
        consolidated_data.append(','.join(current_run_data))

    # Write the output to a new file
    with open(output_file, 'w') as file:
        for line in consolidated_data:
            file.write(line + '\n')

# Specify the input and output file paths
input_file = 'prescales_2016.txt'
output_file = 'processed_prescales_2016.csv'
process_prescales_data(input_file, output_file)

In [6]:
import pickle

def extract_l1_configurations(input_file):
    l1_dict = {}
    with open(input_file, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            run_number = parts[0]
            l1_config = 'OR ' + line.split('OR ')[1].strip()

            if l1_config not in l1_dict:
                l1_dict[l1_config] = set()
            l1_dict[l1_config].add(run_number)

    # Convert sets to lists
    for key in l1_dict:
        l1_dict[key] = list(l1_dict[key])

    return l1_dict

# File paths
input_file = 'processed_prescales_2018.csv'
output_pickle = 'l1_configurations_2018.pickle'

# Extract configurations and save to pickle
l1_configurations = extract_l1_configurations(input_file)
with open(output_pickle, 'wb') as handle:
    pickle.dump(l1_configurations, handle, protocol=pickle.HIGHEST_PROTOCOL)